In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa

import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
import keras
import warnings
from keras.utils import Sequence
warnings.filterwarnings("ignore")
SIZE = 299
SEED = 777
THRESHOLD = 0.2

Using TensorFlow backend.


In [2]:
# Load dataset info
DIR = '../input/'
data = pd.read_csv('../input/train.csv')

# train_dataset_info = []
# for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
#     train_dataset_info.append({
#         'path':os.path.join(path_to_train, name),
#         'labels':np.array([int(label) for label in labels])})
# train_dataset_info = np.array(train_dataset_info)

In [3]:
def getTrainDataset():
    
    path_to_train = DIR + '/train/'
    data = pd.read_csv(DIR + '/train.csv')

    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

def getTestDataset():
    
    path_to_test = DIR + '/test/'
    data = pd.read_csv(DIR + '/sample_submission.csv')

    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)
paths, labels = getTrainDataset()

In [4]:
# credits: https://github.com/keras-team/keras/blob/master/keras/utils/data_utils.py#L302
# credits: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
from random import randint
class ProteinDataGenerator(keras.utils.Sequence):
            
    def __init__(self, paths, labels, batch_size, shape, channels = [], shuffle = False, use_cache = False, augmentor = False):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.use_cache = use_cache
        self.channels = channels
        self.augmentor = augmentor
        self.clahe = cv2.createCLAHE()
        if use_cache == True:
            self.cache = np.zeros((paths.shape[0], shape[0], shape[1], len(channels)))
            self.is_cached = np.zeros((paths.shape[0]))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
        # Generate data
        if self.use_cache == True:
            X = self.cache[indexes]
            for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
                image = self.__load_image(path)
                self.is_cached[indexes[i]] = 1
                self.cache[indexes[i]] = image
                X[i] = image
        else:
            for i, path in enumerate(paths):
                X[i] = self.__load_image(path)
        if self.augmentor == True:
            for i, item in enumerate(X):
                X[i] = self.augment(item)
        y = self.labels[indexes]
        
        return X, y
    
    def on_epoch_end(self):
        
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        images = []
        for channel in self.channels:
            im = np.array(Image.open(path + '_' + channel + '.png'))
            
#             im = clahe.apply(im)
            images.append(im)
            
        if len(self.channels) >= 2:
            im = np.stack((
                images
            ), -1)
            im = cv2.resize(im, (SIZE,SIZE))
            im = np.divide(im, 255)

        else:
            im = images[0]
            im = cv2.resize(im, (SIZE,SIZE))
            im = np.divide(im, 255)
            im = np.expand_dims(im, 2)
        return im
    def augment(self, image):
        if randint(0,1) == 1:
            augment_img = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Flipud(0.5), # horizontal flips
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                        rotate=(-180, 180),
                        shear=(-4, 4)
                    )
                ])], random_order=True)


            image_aug = augment_img.augment_image(image)
            return image_aug
        else:
            return image
    

In [5]:
SHAPE = (299, 299, 4)

In [6]:
# channels = ["red", "green", "blue"]
# for path in paths[0:10]:
#     images = []
#     for channel in channels:
#         im = np.array(Image.open(path + '_' + channel + '.png'))
# #         im = cv2.equalizeHist(im)
#         clahe = cv2.createCLAHE()
#         im = clahe.apply(im)
# #         plt.imshow(im)
#         images.append(im)

#     if len(channels) >= 2:
#         im = np.stack((
#             images
#         ), -1)
#         im = cv2.resize(im, (SIZE,SIZE))
#         im = np.divide(im, 255)
        
        
#     else:
#         im = images[0]
#         im = cv2.resize(im, (SIZE,SIZE))
#         im = np.divide(im, 255)
#         im = np.expand_dims(im, 2)
#     plt.imshow(augment(im))

In [7]:

# class data_generator:
    
#     def create_train(dataset_info, batch_size, shape, augument=True):
#         assert shape[2] == 3
#         while True:
#             dataset_info = shuffle(dataset_info)
#             for start in range(0, len(dataset_info), batch_size):
#                 end = min(start + batch_size, len(dataset_info))
#                 batch_images = []
#                 X_train_batch = dataset_info[start:end]
#                 batch_labels = np.zeros((len(X_train_batch), 28))
#                 for i in range(len(X_train_batch)):
#                     image = data_generator.load_image(
#                         X_train_batch[i]['path'], shape)   
#                     if augument:
#                         image = data_generator.augment(image)
#                     batch_images.append(image/255.)
#                     batch_labels[i][X_train_batch[i]['labels']] = 1
#                 yield np.array(batch_images, np.float32), batch_labels

#     def load_image(path, shape):
#         image_red_ch = Image.open(path+'_red.png')
#         image_yellow_ch = Image.open(path+'_yellow.png')
#         image_green_ch = Image.open(path+'_green.png')
#         image_blue_ch = Image.open(path+'_blue.png')
#         image = np.stack((
#         np.array(image_red_ch), 
#         np.array(image_green_ch), 
#         np.array(image_blue_ch)), -1)
#         image = cv2.resize(image, (shape[0], shape[1]))
#         return image

#     def augment(image):
#         augment_img = iaa.Sequential([
#             iaa.OneOf([
#                 iaa.Affine(rotate=0),
#                 iaa.Affine(rotate=90),
#                 iaa.Affine(rotate=180),
#                 iaa.Affine(rotate=270),
#                 iaa.Fliplr(0.5),
#                 iaa.Flipud(0.5),
#             ])], random_order=True)

#         image_aug = augment_img.augment_image(image)
#         return image_aug

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D, MaxPooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
from keras.utils import multi_gpu_model

In [9]:
def create_model(input_shape, n_out, channels):
    input_tensor = Input(shape=(299,299,len(channels)))

    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=(299,299,3)
                            )
    bn = BatchNormalization()(input_tensor)
    x = Conv2D(3, kernel_size=(1,1), activation='relu', padding = "same")(bn)
    x = base_model(x)
    bn = BatchNormalization()(x)
    x = Conv2D(128, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
#     output = Dense(n_out, activation='sigmoid')(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [10]:
def simple_model(input_shape, n_out, channels):
    input_tensor = Input(shape=(299,299,len(channels)))
    bn = BatchNormalization()(input_tensor)
    x = Conv2D(8, kernel_size=(3,3), activation='relu', padding = "same")(bn)
    x = Conv2D(8, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(16, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = Conv2D(16, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(32, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = Conv2D(32, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(64, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = Conv2D(64, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(128, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = Conv2D(128, kernel_size=(3,3), activation='relu', padding = "same")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(256, kernel_size=(3,3), activation='relu', padding = "valid")(x)
    x = Conv2D(256, kernel_size=(3,3), activation='relu', padding = "valid")(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
#     output = Dense(n_out, activation='sigmoid')(x)
    output = Dense(n_out, activation="sigmoid")(x)
    model = Model(input_tensor, output)
    
    return model

In [11]:
def f1(y_true, y_pred):
    #y_pred = K.round(y_pred)
    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)
def f1_loss(y_true, y_pred):
    
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

In [13]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

epochs = 10; batch_size = 32;VAL_RATIO = .1;DEBUG = False
# split data into train, valid
paths, labels = getTrainDataset()

# divide to 
keys = np.arange(paths.shape[0], dtype=np.int)  
np.random.seed(SEED)
np.random.shuffle(keys)
lastTrainIndex = int((1-VAL_RATIO) * paths.shape[0])

if DEBUG == True:  # use only small subset for debugging, Kaggle's RAM is limited
    pathsTrain = paths[0:256]
    labelsTrain = labels[0:256]
    pathsVal = paths[lastTrainIndex:lastTrainIndex+256]
    labelsVal = labels[lastTrainIndex:lastTrainIndex+256]
    use_cache = True
else:
    pathsTrain = paths[0:lastTrainIndex]
    labelsTrain = labels[0:lastTrainIndex]
    pathsVal = paths[lastTrainIndex:]
    labelsVal = labels[lastTrainIndex:]
    use_cache = False

print(paths.shape, labels.shape)
print(pathsTrain.shape, labelsTrain.shape, pathsVal.shape, labelsVal.shape)
use_cache = True
channels = ["green", "blue", "red", "yellow"]
tg = ProteinDataGenerator(pathsTrain, labelsTrain, batch_size, SHAPE, channels, use_cache=use_cache, augmentor = True)
vg = ProteinDataGenerator(pathsVal, labelsVal, batch_size, SHAPE, channels, use_cache=use_cache, augmentor = False)

(31072,) (31072, 28)
(27964,) (27964, 28) (3108,) (3108, 28)


In [14]:
# create train and valid datagens
# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

In [15]:
checkpoint = ModelCheckpoint('../working/InceptionV3.h5', monitor='val_f1', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = False)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_f1', factor=0.5, patience=10, 
                                   verbose=1, mode='max', epsilon=0.0001)
early = EarlyStopping(monitor="val_f1", 
                      mode="max", 
                      patience=20)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [16]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        pt_1 = K.clip(pt_1, 1e-3, .999)
        pt_0 = K.clip(pt_0, 1e-3, .999)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed
def KerasFocalLoss(target, input):
    
    gamma = 2.
    input = tf.cast(input, tf.float32)
    
    max_val = K.clip(-input, 0, 1)
    loss = input - input * target + max_val + K.log(K.exp(-max_val) + K.exp(-input - max_val))
    invprobs = tf.log_sigmoid(-input * (target * 2.0 - 1.0))
    loss = K.exp(invprobs * gamma) * loss
    
    return K.mean(K.sum(loss, axis=1))

In [17]:
# warm up model
import tensorflow as tf
# with tf.device('/cpu:0'):
model = simple_model(
    input_shape=(SIZE,SIZE,len(channels)), 
    n_out=28, channels = channels)

# for layer in model.layers:
#     layer.trainable = False
# model.layers[1].trainable = True
# model.layers[2].trainable = True
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.layers[-3].trainable = True
# model.layers[-4].trainable = True
# model.layers[-5].trainable = True
# model.layers[-6].trainable = True

model.summary()
# model = multi_gpu_model(model, gpus = 2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 299, 299, 4)       16        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 299, 299, 8)       296       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 299, 299, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 16)      1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 149, 149, 16)      2320      
__________

In [18]:
model.compile(
    loss="binary_crossentropy", 
    optimizer=Adam(1e-03),
    metrics=['binary_accuracy', f1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 4)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 299, 299, 4)       16        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 299, 299, 8)       296       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 299, 299, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 16)      1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 149, 149, 16)      2320      
__________

In [20]:
hist =  model.fit_generator(
        tg,
        steps_per_epoch=np.ceil(float(len(pathsTrain)) / float(batch_size)),
        validation_data=vg,
        validation_steps=np.ceil(float(len(pathsVal)) / float(batch_size)),
        epochs=100, 
        verbose=1,
        callbacks = callbacks_list)

Epoch 1/100
874/874 [==============================] - 353s 403ms/step - loss: 0.1148 - binary_accuracy: 0.9601 - f1: 0.2571 - val_loss: 0.1174 - val_binary_accuracy: 0.9598 - val_f1: 0.2438

Epoch 00001: val_f1 did not improve from 0.25719
Epoch 2/100
874/874 [==============================] - 337s 385ms/step - loss: 0.1165 - binary_accuracy: 0.9597 - f1: 0.2514 - val_loss: 0.1193 - val_binary_accuracy: 0.9585 - val_f1: 0.2388

Epoch 00002: val_f1 did not improve from 0.25719
Epoch 3/100
874/874 [==============================] - 340s 389ms/step - loss: 0.1157 - binary_accuracy: 0.9599 - f1: 0.2537 - val_loss: 0.1176 - val_binary_accuracy: 0.9593 - val_f1: 0.2435

Epoch 00003: val_f1 did not improve from 0.25719
Epoch 4/100
874/874 [==============================] - 313s 358ms/step - loss: 0.1152 - binary_accuracy: 0.9600 - f1: 0.2566 - val_loss: 0.1165 - val_binary_accuracy: 0.9589 - val_f1: 0.2571

Epoch 00004: val_f1 did not improve from 0.25719
Epoch 5/100
874/874 [===============

874/874 [==============================] - 291s 333ms/step - loss: 0.0989 - binary_accuracy: 0.9654 - f1: 0.2994 - val_loss: 0.1134 - val_binary_accuracy: 0.9608 - val_f1: 0.2721

Epoch 00033: val_f1 did not improve from 0.27323
Epoch 34/100
874/874 [==============================] - 325s 372ms/step - loss: 0.0992 - binary_accuracy: 0.9653 - f1: 0.2983 - val_loss: 0.1133 - val_binary_accuracy: 0.9613 - val_f1: 0.2656

Epoch 00034: val_f1 did not improve from 0.27323
Epoch 35/100
874/874 [==============================] - 307s 351ms/step - loss: 0.0982 - binary_accuracy: 0.9657 - f1: 0.2993 - val_loss: 0.1130 - val_binary_accuracy: 0.9609 - val_f1: 0.2693

Epoch 00035: val_f1 did not improve from 0.27323
Epoch 36/100
874/874 [==============================] - 320s 366ms/step - loss: 0.0981 - binary_accuracy: 0.9656 - f1: 0.3020 - val_loss: 0.1132 - val_binary_accuracy: 0.9606 - val_f1: 0.2682

Epoch 00036: val_f1 did not improve from 0.27323
Epoch 37/100
874/874 [=======================

874/874 [==============================] - 303s 346ms/step - loss: 0.0885 - binary_accuracy: 0.9692 - f1: 0.3269 - val_loss: 0.1160 - val_binary_accuracy: 0.9611 - val_f1: 0.2790

Epoch 00065: val_f1 did not improve from 0.27974
Epoch 66/100
874/874 [==============================] - 291s 332ms/step - loss: 0.0881 - binary_accuracy: 0.9694 - f1: 0.3268 - val_loss: 0.1158 - val_binary_accuracy: 0.9609 - val_f1: 0.2701

Epoch 00066: val_f1 did not improve from 0.27974
Epoch 67/100
874/874 [==============================] - 318s 364ms/step - loss: 0.0876 - binary_accuracy: 0.9697 - f1: 0.3299 - val_loss: 0.1186 - val_binary_accuracy: 0.9603 - val_f1: 0.2706

Epoch 00067: val_f1 did not improve from 0.27974
Epoch 68/100
874/874 [==============================] - 300s 343ms/step - loss: 0.0874 - binary_accuracy: 0.9697 - f1: 0.3319 - val_loss: 0.1150 - val_binary_accuracy: 0.9610 - val_f1: 0.2731

Epoch 00068: val_f1 did not improve from 0.27974
Epoch 69/100
874/874 [=======================

874/874 [==============================] - 290s 332ms/step - loss: 0.0686 - binary_accuracy: 0.9767 - f1: 0.3804 - val_loss: 0.1224 - val_binary_accuracy: 0.9611 - val_f1: 0.2793

Epoch 00098: val_f1 did not improve from 0.28296
Epoch 99/100
874/874 [==============================] - 332s 380ms/step - loss: 0.0677 - binary_accuracy: 0.9770 - f1: 0.3836 - val_loss: 0.1222 - val_binary_accuracy: 0.9603 - val_f1: 0.2813

Epoch 00099: val_f1 did not improve from 0.28296
Epoch 100/100
874/874 [==============================] - 291s 333ms/step - loss: 0.0678 - binary_accuracy: 0.9771 - f1: 0.3843 - val_loss: 0.1218 - val_binary_accuracy: 0.9607 - val_f1: 0.2763

Epoch 00100: val_f1 did not improve from 0.28296


In [21]:
# train all layers

for layer in model.layers:
    print(layer)
    layer.trainable = True
model.compile(loss=focal_loss(),
            optimizer=Adam(lr=1e-4),
            metrics=['accuracy', f1])

In [ ]:

batch_size = 12
hist =  model.fit_generator(
        tg,
        steps_per_epoch=np.ceil(float(len(pathsTrain)) / float(batch_size))/2,
        validation_data=vg,
        validation_steps=np.ceil(float(len(pathsVal)) / float(batch_size))/2,
        epochs=200, 
        verbose=1,
        callbacks=callbacks_list)
    

Epoch 1/200
1166/1165 [==============================] - 606s 520ms/step - loss: 13.8179 - acc: 0.4125 - f1: 0.1311 - val_loss: 13.0988 - val_acc: 0.4613 - val_f1: 0.1407

Epoch 00001: val_f1 improved from 0.10443 to 0.14066, saving model to ../working/InceptionV3.h5
Epoch 2/200
1166/1165 [==============================] - 530s 455ms/step - loss: 12.4071 - acc: 0.4781 - f1: 0.1634 - val_loss: 12.4184 - val_acc: 0.5031 - val_f1: 0.1635

Epoch 00002: val_f1 improved from 0.14066 to 0.16350, saving model to ../working/InceptionV3.h5
Epoch 3/200
1166/1165 [==============================] - 554s 475ms/step - loss: 11.2627 - acc: 0.5278 - f1: 0.1934 - val_loss: 12.4274 - val_acc: 0.4550 - val_f1: 0.1691

Epoch 00003: val_f1 improved from 0.16350 to 0.16911, saving model to ../working/InceptionV3.h5
Epoch 4/200
1166/1165 [==============================] - 593s 508ms/step - loss: 9.4958 - acc: 0.5981 - f1: 0.2390 - val_loss: 13.9004 - val_acc: 0.4100 - val_f1: 0.1862

Epoch 00004: val_f1 impro

In [ ]:
# model.compile(loss=f1_loss,
#             optimizer=Adam(lr=1e-4),
#             metrics=['accuracy', f1])
# hist =  model.fit_generator(
#         tg,
#         steps_per_epoch=np.ceil(float(len(pathsTrain)) / float(batch_size))/2,
#         validation_data=vg,
#         validation_steps=np.ceil(float(len(pathsVal)) / float(batch_size))/2,
#         epochs=200, 
#         verbose=1,
#         callbacks=callbacks_list)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].set_title('loss')
ax[0].plot(hist.epoch, hist.history["loss"], label="Train loss")
ax[0].plot(hist.epoch, hist.history["val_loss"], label="Validation loss")
ax[1].set_title('acc')
ax[1].plot(hist.epoch, hist.history["f1"], label="Train F1")
ax[1].plot(hist.epoch, hist.history["val_f1"], label="Validation F1")
ax[0].legend()
ax[1].legend()

In [ ]:
from keras.models import load_model
bestModel = load_model('../working/InceptionV3.h5', custom_objects={'f1': f1, 'f1_loss': f1_loss, 'focal_loss_fixed':focal_loss()})

In [ ]:
from tqdm import tqdm
lastFullValPred = np.empty((0, 28))
lastFullValLabels = np.empty((0, 28))
for i in tqdm(range(len(vg))): 
    im, lbl = vg[i]
    scores = bestModel.predict(im)
    lastFullValPred = np.append(lastFullValPred, scores, axis=0)
    lastFullValLabels = np.append(lastFullValLabels, lbl, axis=0)
print(lastFullValPred.shape, lastFullValLabels.shape)

In [ ]:
from sklearn.metrics import f1_score as off1
rng = np.arange(0, 1, 0.001)
f1s = np.zeros((rng.shape[0], 28))
for j,t in enumerate(tqdm(rng)):
    for i in range(28):
        p = np.array(lastFullValPred[:,i]>t, dtype=np.int8)
        scoref1 = off1(lastFullValLabels[:,i], p, average='binary')
        f1s[j,i] = scoref1

In [ ]:
print('Individual F1-scores for each class:')
print(np.max(f1s, axis=0))
print('Macro F1-score CV =', np.mean(np.max(f1s, axis=0)))
plt.plot(rng, f1s)
T = np.empty(28)
for i in range(28):
    T[i] = rng[np.where(f1s[:,i] == np.max(f1s[:,i]))[0][0]]
print('Probability threshold maximizing CV F1-score for each class:')
print(T)

In [ ]:
pathsTest, labelsTest = getTestDataset()

testg = ProteinDataGenerator(pathsTest, labelsTest, batch_size, SHAPE, channels)
submit = pd.read_csv(DIR + '/sample_submission.csv')
P = np.zeros((pathsTest.shape[0], 28))
for i in tqdm(range(len(testg))):
    images, labels = testg[i]
    score = bestModel.predict(images)
    P[i*batch_size:i*batch_size+score.shape[0]] = score

In [ ]:
PP = np.array(P)

In [ ]:
prediction = []

for row in tqdm(range(submit.shape[0])):
    
    str_label = ''
    
    for col in range(PP.shape[1]):
        if(PP[row, col] < T[col]):
            str_label += ''
        else:
            str_label += str(col) + ' '
    prediction.append(str_label.strip())
    
submit['Predicted'] = np.array(prediction)
submit.to_csv('transfer_1x1conv_aug_focal_loss.csv', index=False)

In [ ]:
# from tqdm import tqdm
# pathsTest, labelsTest = getTestDataset()

# testg = ProteinDataGenerator(pathsTest, labelsTest, batch_size, SHAPE)
# submit = pd.read_csv(DIR + '/sample_submission.csv')
# P = np.zeros((pathsTest.shape[0], 28))
# for i in tqdm(range(len(testg))):
#     images, labels = testg[i]
#     score = bestModel.predict(images)
#     P[i*batch_size:i*batch_size+score.shape[0]] = score

In [ ]:
# PP = np.array(P)

In [ ]:
# prediction = []

# for row in tqdm(range(submit.shape[0])):
    
#     str_label = ''
    
#     for col in range(PP.shape[1]):
#         if(PP[row, col] < .2):   # to account for losing TP is more costly than decreasing FP
#             #print(PP[row])
#             str_label += ''
#         else:
#             str_label += str(col) + ' '
#     prediction.append(str_label.strip())
    
# submit['Predicted'] = np.array(prediction)
# submit.to_csv('datagenerator_model_v2.csv', index=False)